In [1]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List, Tuple

import hydra
import lightning as pl
from omegaconf import DictConfig
from lightning import Callback, LightningDataModule, LightningModule, Trainer
from lightning.pytorch.loggers import Logger

from terralearn import utils

import torch
import hydra
import omegaconf
import pyrootutils
import pandas as pd
from tqdm import tqdm
import numpy as np

In [13]:
df = pd.read_csv('/home/ubuntu/FGVC11_2024/data/PlantTrait/test.csv')
df['path'] = '/home/ubuntu/FGVC11_2024/data/PlantTrait/test_images/' + df['id'].astype(str) + '.jpeg'
# df.to_csv('/home/ubuntu/FGVC11_2024/data/PlantTrait/df_test.csv', index=False)

In [7]:
cfg = omegaconf.OmegaConf.load("/home/ubuntu/FGVC11_2024/configs/data/plant_traits_data.yaml")
datamodule: LightningDataModule = hydra.utils.instantiate(cfg)
datamodule.setup()

In [8]:
for batch in datamodule.test_dataloader():
    break

In [9]:
cfg = omegaconf.OmegaConf.load("/home/ubuntu/FGVC11_2024/configs/model/plant_traits_model.yaml")
model = hydra.utils.instantiate(cfg)

Using cache found in /home/ubuntu/.cache/torch/hub/facebookresearch_dinov2_main
/home/ubuntu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/ubuntu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/ubuntu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")
/home/ubuntu/miniconda3/envs/transformers/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:196: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(


In [11]:
model.load_from_checkpoint('/home/ubuntu/FGVC11_2024/logs/train/runs/2024-04-04_08-39-28/checkpoints/epoch_058.ckpt')

PlantTraitModule(
  (model): PlantCNN(
    (body): DinoVisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (blocks): ModuleList(
        (0-11): 12 x NestedTensorBlock(
          (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (attn): MemEffAttention(
            (qkv): Linear(in_features=384, out_features=1152, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=384, out_features=384, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=384, out_features=1536, bias=True)
            (act): GELU(approximate='none')
            (fc2): Linear(in_features=1536, out_features=384, bias=

In [12]:
device = 'cuda'
model.to(device)

PlantTraitModule(
  (model): PlantCNN(
    (body): DinoVisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (blocks): ModuleList(
        (0-11): 12 x NestedTensorBlock(
          (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (attn): MemEffAttention(
            (qkv): Linear(in_features=384, out_features=1152, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=384, out_features=384, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=384, out_features=1536, bias=True)
            (act): GELU(approximate='none')
            (fc2): Linear(in_features=1536, out_features=384, bias=

In [14]:
all_predictions = []
for batch in tqdm(datamodule.test_dataloader()):
    # Unpack the batch
    images = batch["image"]

    # Move data to the device
    images = images.to(device)

    # Make predictions
    with torch.no_grad():
        predictions = model.forward(images)
    # break

    # Move predictions back to CPU if necessary
    predictions = predictions.cpu().numpy()

    # Append predictions to the list
    all_predictions.append(predictions)

# Concatenate predictions from all batches
all_predictions = np.concatenate(all_predictions, axis=0)

# Create a DataFrame with the predictions and corresponding IDs
submission_df = pd.DataFrame({
    'id': df['id'].values,
    'X4': all_predictions[:, 0],
    'X11': all_predictions[:, 1],
    'X18': all_predictions[:, 2],
    'X50': all_predictions[:, 4],
    'X26': all_predictions[:, 3],
    'X3112': all_predictions[:, 5],
})

100%|██████████| 410/410 [00:50<00:00,  8.19it/s]


In [19]:
images.shape

torch.Size([1, 3, 490, 490])

In [18]:
submission_df.describe()

,id,X4,X11,X18,X50,X26,X3112
count,6.545000e+03,6545.000000,6545.000000,6545.000000,6545.000000,6545.000000,6545.000000
mean,1.930247e+08,0.289149,-0.181909,-0.279819,0.006340,-0.044360,-0.255776
std,2.601377e+07,0.156600,0.153120,0.146321,0.157188,0.142666,0.156738
min,1.040495e+06,-0.269778,-0.823464,-0.914760,-0.551631,-0.550513,-0.855559
25%,1.971690e+08,0.187059,-0.285264,-0.377304,-0.099288,-0.137000,-0.359471
50%,2.008276e+08,0.292193,-0.184891,-0.281518,0.005224,-0.040999,-0.253713
75%,2.022672e+08,0.393154,-0.080483,-0.181248,0.112426,0.054326,-0.150215
max,2.030654e+08,0.951173,0.478563,0.255759,0.638275,0.413662,0.300938


In [16]:
submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df.shape

In [ ]:
import torchsummary

In [ ]:
import timm
import torch.nn as nn
class PlantCNN(nn.Module):
    def __init__(self, num_targets=6):
        super(PlantCNN, self).__init__()
        self.train_tokens = False
        self.trainable_backbone_layers = 4
        self.body = torch.hub.load(
            "facebookresearch/dinov2", "dinov2_vits14_reg", pretrained=True
        )
        for i, layer in enumerate([self.body.patch_embed, self.body.norm]):
            for p in layer.parameters():
                p.requires_grad = False

        if not self.train_tokens:
            self.body.cls_token.requires_grad = False
            self.body.pos_embed.requires_grad = False
            self.body.register_tokens.requires_grad = False
            self.body.mask_token.requires_grad = False

        if self.trainable_backbone_layers is not None:
            for i in range(0, len(self.body.blocks) - self.trainable_backbone_layers):
                for p in self.body.blocks[i].parameters():
                    p.requires_grad = False
        self.relu1 = nn.ReLU()
        self.fc1 = nn.Linear(
            self.body.num_features, self.body.num_features // 2
        )
        self.relu2 = nn.ReLU()
        self.fc2 = nn.Linear(self.body.num_features // 2, num_targets)

    def forward(self, x):
        x = self.body(x)
        x = self.relu1(x)
        x = self.fc1(x)
        x = self.relu2(x)
        x = self.fc2(x)
        return x

model = PlantCNN().to('cuda')

In [ ]:
torchsummary.summary(model, (3, 490, 490))